# Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train three machine learning classifiers: CART, Support Vector Machine and Random Forest. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>
**NOTE:** The input image needs to have the bands B1, B2, B3, B4, B5, B8, B11, B12 to apply masks correctly. The classifications will use only the bands B1, B2, B3, B4 and Blue/Green, which represent the bands that penetrates the most into the water column.

Script by: Luis Lizcano-Sandoval<br/>
College of Marine Sciences, University of South Florida<br/>
Updated: 11/16/2020

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds, land, and deep areas >20m
3. Apply Depth-Invariant Index (generates band-ratios B1B2, B1B3, B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (CART, SVM and RF)
6. Get confusion matrices and accuracies
7. Export output to EE Assets (.tiff)
8. Save matrices in local computer (.xlxs)

**Load required libraries:**

In [1]:
import ee
import pandas as pd
import xlsxwriter
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
import datetime
from functions import CloudScore6S,landMaskFunction,DII

ee.Initialize()

In [2]:
print(ee.__version__)

0.1.257


**Insert the ID of the image of interest and folder from the personal Assets:**

In [3]:
## Select set of images belonging to a same satellite sensor and year.

## Sentinel 2
imageList = [
'20210314T155929_20210314T160346_T17RLL',
'20210413T155819_20210413T161213_T17RLL',
'20210309T160101_20210309T161423_T17RLM',
'20210106T161649_20210106T162243_T17RLM',
'20210309T160101_20210309T161423_T17RLM'
]

## Landsat list
# 2019
# imageList = [
# 'LE07_017040_20190223',
# 'LE07_017040_20191005']

# 2000
# imageList = [
# 'LE07_017040_20000914',
# 'LT05_017040_20000602']

# 1990
# imageList = [
# 'LT05_017041_20101121',
# 'LT05_017041_20101004',
# 'LT05_017041_20101020',
# 'LT05_017041_20090118',
# 'LT05_017041_20110313'
# ]

# for i in range(len(imageList)):
#     print(imageList[i])

**Some metadata:**

In [4]:
#boaFolder = 'FL_10'
exportFolder = 'FL_21'
dataFolder = 'Ground-points-19'
#dataFolder = 'Ground-points-00'
#dataFolder = 'TB_10' ##Ground-truth data
smoothStr = '_raw_' #Smooth or not? '_smooth_' or '_raw_'

## Image Collection - For use only if BOA Folder is active.
#collection = ee.ImageCollection("users/lizcanosandoval/BOA/Sentinel/"+boaFolder)
#collection = ee.ImageCollection("users/lizcanosandoval/BOA/Landsat/"+boaFolder)

###########################    IMPORT TRAINING DATA    #########################

# Sandy areas
sand_areas = ee.FeatureCollection("users/lizcanosandoval/ground-points/Sand")

# Ground-Points
groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/"+dataFolder)
#groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/Turkey_2020")


## IMPORT OTHER COLLECTIONS:

## ETOPO1: Global 1 Arc-Minute Elevation:
#etopo = ee.Image("NOAA/NGDC/ETOPO1")
#etopo = etopo.select('bedrock') ## Select the bathymetry band

## Florida Bathymetry [NOAA-90m res]
#bathymetry = ee.Image("users/lizcanosandoval/Bathymetry_FL")

## Florida Land [from GADM-HiRes]
#gadm_FL = ee.FeatureCollection("users/lizcanosandoval/gadm36_FL")
gadm_FL = ee.FeatureCollection("users/lizcanosandoval/Florida_10m") ##Created from NDWI using Sentinel-2 imagery
#gadm_FL = ee.FeatureCollection("users/lizcanosandoval/Turkey_gadm")
#gadm_FL = ee.ImageCollection("users/lizcanosandoval/WaterMask_Turkey")


## FAO Global Administrative Unit Layers 2015, Country Boundaries
#FAO = ee.FeatureCollection("FAO/GAUL/2015/level0")

## Filter country. To select country by GAUL code see: http://www.fao.org/countryprofiles/iso3list/en/
#country = FAO.filter(ee.Filter.eq('ADM0_CODE',256))

## Florida's Seagrass Habitats (FWC database)
#seagrass = ee.FeatureCollection("users/lizcanosandoval/Seagrass_Habitat_Florida"),

## Sentinel-2 Tiles over seagrass areas:
#tiles = ee.FeatureCollection("users/lizcanosandoval/S2_tiles_surf-to-shallow_line")
print('Collections loaded')

Collections loaded


**Start classification loop:**

In [5]:
%%time
## Initiate loop:
for i in range(len(imageList)):
    print('Initiating...')
    imageID = imageList[i]
    
    print('Preparing image '+imageID)
    


    ## Filter collection by image ID:
#     imageTarget = collection.filter(ee.Filter.eq('file_id',imageID)).first() ## for use if BOA folder is used
    ##Settings if SR collection is used instead of BOA Folder:
    imageTarget = ee.Image('COPERNICUS/S2_SR/'+imageID)
    imageTarget = imageTarget.divide(10000).set(imageTarget.toDictionary(imageTarget.propertyNames()))
    imageSat = imageTarget.get('SPACECRAFT_NAME').getInfo()
    imageTile = imageTarget.get('MGRS_TILE').getInfo()
    ee_date = imageTarget.get('GENERATION_TIME').getInfo()
    imageDate = str(datetime.datetime.utcfromtimestamp(ee_date/1000.0))
    
    ## more settings and metadata (if images are in BOA folder-GEE Assets)
    imageGeometry = imageTarget.geometry() #Tile geometry. For use always.
#     imageSat = imageTarget.get('satellite').getInfo() #Image satellite
#     imageTile = imageTarget.get('tile_id').getInfo() #Image tile id
#     imageDate = imageTarget.get('date').getInfo() #Image date

    if 'Sentinel' in imageSat:
        imageScale = 10 # Sentinel resolution
    else:
        imageScale = 30 # Landsat resolution

    
    
    ###########################    PREPARE BATHYMETRY DATA    #########################

#     ## Mask depth ranges from the Etopo collection [from -25 to 1 meter]
#     etopo_masked = ee.Image(etopo).updateMask(etopo.lt(1).And(etopo.gt(-25)))

#     ## Resample ETOPO
#     etopo_resample = etopo_masked.reproject(**{\
#                                              'crs': 'EPSG:32617',
#                                              'scale': 90,
#                                             })

#     ## Define a boxcar or low-pass kernel.
#     kernel = ee.Kernel.square(**{\
#                                'radius': 3, 
#                                'units': 'pixels', 
#                                'normalize': True,
#                               })

#     ## Smooth raster
#     #etopo_kernel = etopo_resample.resample('bilinear')
#     etopo_kernel = etopo_resample.convolve(kernel)

#     ## Mask the FL bathymetry collection (NOAA):
#     ## Mask depth ranges from the FL bathymetry collection [Mask depth range from -20 to 2 meters]
#     bathy_masked = ee.Image(bathymetry).updateMask(bathymetry.lt(2).And(bathymetry.gt(-25)))

#     ## Clip bathymetry layers to tile geometry
#     bathyBand = bathy_masked.clip(imageGeometry) ##Florida bathymetry clip (90m res)
#     etopo_clip = etopo_kernel.clip(imageGeometry) ##Etopo clip + kernel (90m resampled)

#     ## Vectorize (to FeatureCollection of polygons) the bathymetry collection,
#     ## which is a raster image.
#     bathyVector = bathyBand.toByte().reduceToVectors(**{
#       'reducer': ee.Reducer.countEvery(),
#       'crs': 'EPSG:4326',
#       'geometry': None,
#       'eightConnected': False,
#       'labelProperty': 'bathymetry',
#       'scale': 1000,
#       'geometryType': 'polygon',
#       'maxPixels': 1e9
#     })

#     ## Extract the bathymetry values from the Etopo layer corresponding to the gap 
#     ## found in the FL Bathymetry Data.
#     bathyOutside = ee.Image.constant(1).clip(bathyVector).mask().Not()
#     bathyGap = etopo_clip.updateMask(bathyOutside)
#     bathyVectorGap = bathyGap.toByte().reduceToVectors(**{
#       'reducer': ee.Reducer.countEvery(),
#       'crs': 'EPSG:4326',
#       'geometry': None,
#       'eightConnected': False,
#       'labelProperty': 'bathymetry',
#       'scale': 1000,
#       'geometryType': 'polygon',
#       'maxPixels': 1e9
#     });
#     bathyVectorGap = bathyVectorGap.geometry().buffer(50) ##Fill gap and extend a buffer of 50m
#     bathyOutside2 = ee.Image.constant(1).clip(bathyVectorGap).mask()
#     bathyGap2 = etopo_clip.updateMask(bathyOutside2) ##Clip the gap in the ETOPO collection.

#     #Finally, fill the gap in the NOAA bathymetry dataset. This is the new NOAA's bathymetry raster layer:
#     bathyFilled = ee.ImageCollection([bathyBand,bathyGap2.select(['bedrock'],['b1'])]).mosaic()


    ###########################    CLOUD MASK    #########################

    ## Recommended Threshold values for
    ## *Sentinel: 2
    ## *Landsat: 5
    if 'Sentinel' in imageSat:
        threshold = 5
    else:
        threshold = 5

    ## Apply cloud mask
    cloudMask = CloudScore6S(imageSat, imageTarget, threshold)


    ###########################    LAND MASK    #########################

    ## Apply land mask
    landMask = landMaskFunction(cloudMask, gadm_FL)
    #landMask = cloudMask.updateMask(gadm_FL.mosaic())


    ###########################    BATHYMETRY MASK    #########################

    ## Apply bathymetry mask
    bathyMask = landMask#.clip(bathyVector) ##Using the NOAA dataset: bathyVector


    ###########################    TURBIDITY MASK    #########################

    ## Turbidity masking is based on the red band reflectances. Based on own observations, values higher than 0.02-0.03 
    ## indicates turbid waters, but sometimes may indicate shallow seagrass banks. So the below algorithm try to separate 
    ## shallow seagrass from turbidity.

    ## Set parameter values
    if 'Sentinel' in imageSat:
        red_band = 'B4' #Red seems work better in this case than B5.
        red_thr_inf = 0.025
        red_thr_sup = 0.2
        green_band = 'B3'
        green_thr = 0.15
        blue_band = 'B2'
        blue_thr = 0.11
    elif 'Landsat8' in imageSat:
        red_band = 'B4'
        red_thr_inf = 0.025
        red_thr_sup = 0.2
        green_band = 'B3'
        green_thr = 0.15
        blue_band = 'B2'
        blue_thr = 0.11
    else:
        red_band = 'B3'
        red_thr_inf = 0.03 #Landsat5/7 are less sensitive
        red_thr_sup = 0.2
        green_band = 'B2'
        green_thr = 0.15
        blue_band = 'B1'
        blue_thr = 0.11

    ## Select the red band
    selectRedBand = bathyMask.select('B4')

    ## Identify turbid areas first (pixel values higher than the threshold)
    turbidMask = selectRedBand.gt(red_thr_inf)

    ## Apply thresholds on red, green and blue bands
    maskRed = selectRedBand.gt(red_thr_inf).And(selectRedBand.lt(red_thr_sup))
    imageMaskRed = bathyMask.mask(maskRed)
    maskGreen = imageMaskRed.select(green_band).lt(green_thr)
    imageMaskGreen = imageMaskRed.mask(maskGreen)
    maskBlue = imageMaskGreen.select(blue_band).lt(blue_thr) ##Shallow seagrass

    ## Final mask (excluding seagrass/including turbid water)
    turbidImage = bathyMask.mask(turbidMask) ## Turbidity
    seagrassImage = bathyMask.mask(maskBlue).mask().Not() ## Shallow seagrass (inverse mask)
    excludeSeagrass = turbidImage.updateMask(seagrassImage) ## Turbidity minus shallow seagrass
    finalMaskImage = bathyMask.mask(excludeSeagrass)
    finalMask = finalMaskImage.mask().Not()

    ## Final Image
    finalImage = bathyMask.updateMask(finalMask)
    print('Image masked...')

    ###########################    WATER COLUMN CORRECTION    #########################    

    ## Filter sand polygons by tile/area:
    #sand = ee.FeatureCollection(sand_areas).flatten().filterBounds(imageGeometry)
    sand = ee.FeatureCollection(sand_areas).filterBounds(imageGeometry)

    ## Run the Depth-Invariant Index Function
    imageDII = DII(finalImage, imageScale, sand)

    print('Depth-Invariant index applied...')

    ###########################    SAMPLING    #########################
    # Classes are:

    # 0: Softbottom
    # 1: Hardbottom
    # 2: Seagrass
    # 3: Sparse seagrass //if available

    ## Filter ground points by tile geometry and display classes
    filterPoints = ee.FeatureCollection(groundPoints).filterBounds(imageGeometry)

    ## Select bands to sample. The B/G band is B2B3 in Sentinel-2 and Landsat-8, and B1B2 for Landsat-7/5
    if 'Sentinel' in imageSat or 'Landsat8' in imageSat:
        bandsClass = ['B1','B2', 'B3', 'B4','B2B3']
        bg = ['B2B3']
    else:
        bandsClass = ['B1','B2', 'B3', 'B1B2']
        bg = ['B1B2']

    ## Add bands of interest to sample training points:
    imageClassify = finalImage.addBands(imageDII.select(bg)).select(bandsClass)
    ##Other configurations:
    #imageClassify = imageDII.select(bandsClass); // Depth invariant Bands B1B2,B1B3,B2B3.
    #imageClassify = imageKD.select(bandsClass)//.addBands(bathyFilled); //KD corr B1-B4, with or without bathymetry.


    

    ###########################    MAKE TRAINING WITHIN THE 5KM BOUNDS OF SEAGRASS HABITATS    #########################
    ## Focuses on the known seagrass areas including a buffer area of 5 km.
    seagrass_buffer = ee.FeatureCollection("users/lizcanosandoval/Seagrass_Habitat_Florida_buff5k")
#     shallowMask = ee.ImageCollection("users/lizcanosandoval/Shallow_Mask_FL")
#     specificTurbidMask = ee.ImageCollection("users/lizcanosandoval/TurbidityMask")
#     listShallowMask = shallowMask.aggregate_array('system:index').getInfo()
#     listTurbidMask = specificTurbidMask.aggregate_array('system:index').getInfo()

    imageClassify = imageClassify.clip(seagrass_buffer)
    
#     ##If image has a specific shallow water mask then apply it:
#     if imageTile in listShallowMask:
#         imageClassify = imageClassify.updateMask(shallowMask.mosaic())

#     ##If image has a specific turbidity mask then apply it:
#     if imageID in listTurbidMask:
#         filterTurbidMask = ee.Image(specificTurbidMask.filter(ee.Filter.eq('system:index',imageID)).first())
#         mask = filterTurbidMask.Not()
#         imageClassify = imageClassify.updateMask(mask)

#     #buffer = ee.FeatureCollection("users/lizcanosandoval/Turkey_coastline_buffer_1km")
#     #imageClassify = imageClassify.clip(buffer)
#     clipper = ee.FeatureCollection("users/lizcanosandoval/ground-points/AOI_SpringsCoast")
#     imageClassify = imageClassify.clip(clipper)
    

    ###########################    APPLY SMOOTHER    #########################
    ## Define a boxcar or low-pass kernel (Used if want to smooth the image)
    smooth = ee.Kernel.euclidean(**{
        'radius': 1, 
        'units': 'pixels', 
        'normalize': True
    })
    
    ## Apply smoother if set:
    if 'smooth' in smoothStr:
        imageClassify = imageClassify.convolve(smooth)
    

    ###########################    GET TRAINING AND VALIDATION DATA    #########################

    ## Sample multi-spectral data using all ground points.
    samplingData = imageClassify.sampleRegions(**{
        'collection': filterPoints,
        'properties': ['class'],
        'scale': imageScale})

    ## Add random numbers to each feature (from 0 to 1).
    randomData = samplingData.randomColumn("random",0)

    ## Split ground data in training (~70%) and validation (~30%) points
    trainingData = randomData.filter(ee.Filter.lt("random",0.7))
    validationData = randomData.filter(ee.Filter.gte("random", 0.7))



    ###########################    TRAIN MODELS AND CLASSIFY    #########################
    print('Training models and classifying...')
    ## Train CART classifier
    trainCART = ee.Classifier.smileCart().train(trainingData,'class',bandsClass)

    ## Train SVM classifier
    SVM = ee.Classifier.libsvm(**{
       'kernelType': 'RBF',
       'gamma': 100,
       'cost': 100
    })
    trainSVM = SVM.train(**{
       'features': trainingData,
       'classProperty': 'class',
       'inputProperties': bandsClass
    })

    ## Train RF classifier
    trainRF = ee.Classifier.smileRandomForest(20).train(trainingData, 'class', bandsClass)

    #### Classify the image using the trained classifier
    classifiedCART = imageClassify.classify(trainCART)
    classifiedSVM = imageClassify.classify(trainSVM)
    classifiedRF = imageClassify.classify(trainRF)
    
    ######### Load seagrass mask 2019 and clip, to clean #########
    seagrass_mask = ee.ImageCollection("users/lizcanosandoval/Seagrass/SeagrassMask_FL_50m").mosaic()
    classifiedCART = classifiedCART.updateMask(seagrass_mask)
    classifiedSVM = classifiedSVM.updateMask(seagrass_mask) #For raster
    classifiedRF = classifiedRF.updateMask(seagrass_mask)


    ###########################    TRAINING ACCURACIES    #########################
    print('Getting accuracies...')
    ## Get a confusion matrix representing resubstitution accuracy.
    ## {Resubstitution error is the error of a model on the training data.}
    ## Axis 0 (first level) of the matrix correspond to the input classes (columns), 
    ## and axis 1 (second level) to the output classes (rows).
    matrixTrainingCART = trainCART.confusionMatrix()
    matrixTrainingSVM = trainSVM.confusionMatrix()
    matrixTrainingRF = trainRF.confusionMatrix()


    ###########################    VALIDATION ACCURACIES    #########################

    ## Calculate accuracy using validation data
    ## Classify the image using the trained classifier
    validationCART = validationData.classify(trainCART)
    validationSVM = validationData.classify(trainSVM)
    validationRF = validationData.classify(trainRF)

    ## Get a confusion matrix representing expected accuracy (Using validation points - 30%), where:
    #  0: Softbottom
    #  1: Hardbottom
    #  2: Dense Seagrass
    #  3: Spare Seagrass

    ## Axis 0 (the rows) of the matrix correspond to the actual values, 
    ## and Axis 1 (the columns) to the predicted values.
    errorMx = {'actual': 'class', 'predicted': 'classification'}
    errorMatrixCART = validationCART.errorMatrix(**errorMx)
    errorMatrixSVM = validationSVM.errorMatrix(**errorMx)
    errorMatrixRF = validationRF.errorMatrix(**errorMx)


    ###########################    USER/PRODUCER ACCURACIES    #########################

    ## Estimate user and producer accuracies
    producerAccuracyCART = errorMatrixCART.producersAccuracy()
    producerAccuracySVM = errorMatrixSVM.producersAccuracy()
    producerAccuracyRF = errorMatrixRF.producersAccuracy()

    # USER
    userAccuracyCART = errorMatrixCART.consumersAccuracy()
    userAccuracySVM = errorMatrixSVM.consumersAccuracy()
    userAccuracyRF = errorMatrixRF.consumersAccuracy()


    ###########################    KAPPA COEFFICIENTS    #########################

    # The Kappa Coefficient is generated from a statistical test to evaluate the accuracy 
    # of a classification. Kappa essentially evaluate how well the classification performed 
    # as compared to just randomly assigning values, i.e. did the classification do better 
    # than random. The Kappa Coefficient can range from -1 to 1. A value of 0 indicated that 
    # the classification is no better than a random classification. A negative number 
    # indicates the classification is significantly worse than random. A value close to 1 
    # indicates that the classification is significantly better than random.
    kappaCART = errorMatrixCART.kappa()
    kappaSVM = errorMatrixSVM.kappa()
    kappaRF = errorMatrixRF.kappa()



    ###########################    EXPORT CLASSIFIED IMAGES    #########################

    # Set the scale properly
    scale = []
    sat = []
    method = ['CART','SVM','RF']
    classifiedCollection = ee.ImageCollection([classifiedCART,classifiedSVM,classifiedRF])
    classifiedList = classifiedCollection.toList(classifiedCollection.size())
    classifiedSize = classifiedList.size().getInfo()
    print('Exporting classified images to EE Assets...')

    for i in range(classifiedSize):

        # Rename satellite
        if 'Sentinel' in imageSat:
            sat = 'Sentinel'
        else:
            sat = 'Landsat'

        ## Select image
        image = ee.Image(classifiedList.get(i))

        # set some properties for export
        output = image.set({'satellite': imageSat,
                       'tile_id': imageTile,
                       'file_id': imageID,                                               
                       'date': imageDate,
                       'year': imageDate[0:4],
                       'classifier': method[i],
                       'generator': 'Lizcano-Sandoval',
                            })

        # define YOUR assetID. (This do not create folders, you need to create them manually)
        assetID = 'users/lizcanosandoval/Seagrass/'+sat+'/'+exportFolder+'/' ##This goes to an ImageCollection folder
        fileName = imageID+smoothStr+ method[i]
        path = assetID + fileName

        ## Batch Export to Assets
        ee.batch.Export.image.toAsset(\
            image = ee.Image(output),                                                    
            description = method[i] +smoothStr+ imageID,
            assetId = path,
            region = imageGeometry.buffer(10),                                      
            maxPixels = 1e13,
            scale = imageScale).start()
        print('Classified Image '+str(i+1)+': '+imageID +smoothStr+ method[i]+' submitted...')
    print('Classified images submitted!')



    ###########################    SAVE MATRICES TO WORKING DIRECTORY    #########################
    print('Saving matrices to working directory...')
    # Extract values from each matrix
    CART_trainingMatrix = matrixTrainingCART.array().getInfo()
    CART_trainingAccuracy = matrixTrainingCART.accuracy().getInfo()
    CART_errorMatrix = errorMatrixCART.array().getInfo()
    CART_errorAccuracy = errorMatrixCART.accuracy().getInfo()
    CART_producerAccuracy = producerAccuracyCART.getInfo()
    CART_userAccuracy = userAccuracyCART.getInfo()
    CART_kappa = kappaCART.getInfo()
    SVM_trainingMatrix = matrixTrainingSVM.array().getInfo()
    SVM_trainingAccuracy = matrixTrainingSVM.accuracy().getInfo()
    SVM_errorMatrix = errorMatrixSVM.array().getInfo()
    SVM_errorAccuracy = errorMatrixSVM.accuracy().getInfo()
    SVM_producerAccuracy = producerAccuracySVM.getInfo()
    SVM_userAccuracy = userAccuracySVM.getInfo()
    SVM_kappa = kappaSVM.getInfo()
    RF_trainingMatrix = matrixTrainingRF.array().getInfo()
    RF_trainingAccuracy = matrixTrainingRF.accuracy().getInfo()
    RF_errorMatrix = errorMatrixRF.array().getInfo()
    RF_errorAccuracy = errorMatrixRF.accuracy().getInfo()
    RF_producerAccuracy = producerAccuracyRF.getInfo()
    RF_userAccuracy = userAccuracyRF.getInfo()
    RF_kappa = kappaRF.getInfo()

    ## Convert matrices to pandas dataframes:
    #Training Matrices
    rowIndex = {0:'Sb', 1:'Hb', 2:'Dn', 3:'Sp'}
    TM_CART = pd.DataFrame(CART_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
    TM_SVM = pd.DataFrame(SVM_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
    TM_RF = pd.DataFrame(RF_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
    TM_concat = pd.concat([TM_CART, TM_SVM, TM_RF], keys=['CART','SVM','RF'])

    #Training Accuracies
    TA_CART = pd.Series(CART_trainingAccuracy)
    TA_SVM = pd.Series(SVM_trainingAccuracy)
    TA_RF = pd.Series(RF_trainingAccuracy)
    TA_concat = pd.DataFrame(pd.concat([TA_CART, TA_SVM, TA_RF],ignore_index=True), columns=(['Tr_Accuracy']))\
                    .rename({0:'CART',1:'SVM',2:'RF'})

    #Validation-Error Matrices
    VM_CART = pd.DataFrame(CART_errorMatrix).rename(columns=rowIndex, index=rowIndex)
    VM_SVM = pd.DataFrame(SVM_errorMatrix).rename(columns=rowIndex, index=rowIndex)
    VM_RF = pd.DataFrame(RF_errorMatrix).rename(columns=rowIndex, index=rowIndex)
    VM_concat = pd.concat([VM_CART, VM_SVM, VM_RF], keys=['CART','SVM','RF'])

    #Validation Accuracies
    VA_CART = pd.Series(CART_errorAccuracy)
    VA_SVM = pd.Series(SVM_errorAccuracy)
    VA_RF = pd.Series(RF_errorAccuracy)
    VA_concat = pd.DataFrame(pd.concat([VA_CART, VA_SVM, VA_RF],ignore_index=True), columns=(['Va_Accuracy']))\
                    .rename({0:'CART',1:'SVM',2:'RF'})

    #Producer-User Accuracies
    ## Create a pandas dataframe with producer and user accuracies:
    dfPA_CART = pd.DataFrame(producerAccuracyCART.getInfo(), columns=['Producer'])
    dfUA_CART = pd.DataFrame(userAccuracyCART.getInfo()).transpose()
    dfPA_SVM = pd.DataFrame(producerAccuracySVM.getInfo(), columns=['Producer'])
    dfUA_SVM = pd.DataFrame(userAccuracySVM.getInfo()).transpose()
    dfPA_RF = pd.DataFrame(producerAccuracyRF.getInfo(), columns=['Producer'])
    dfUA_RF = pd.DataFrame(userAccuracyRF.getInfo()).transpose()

    PU_CART = pd.concat([dfPA_CART, dfUA_CART.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)
    PU_SVM = pd.concat([dfPA_SVM, dfUA_SVM.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)
    PU_RF = pd.concat([dfPA_RF, dfUA_RF.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)
    PU_concat = pd.concat([PU_CART, PU_SVM, PU_RF], keys=['CART','SVM','RF'])

    # Kappa coefficients
    Kp_CART = pd.Series(CART_kappa)
    Kp_SVM = pd.Series(SVM_kappa)
    Kp_RF = pd.Series(RF_kappa)
    Kp_concat = pd.DataFrame(pd.concat([Kp_CART, Kp_SVM, Kp_RF],ignore_index=True), columns=(['Kappa']))\
                    .rename({0:'CART',1:'SVM',2:'RF'})

    # Extract the number of training and validation points per class:
    trainingInfo = trainingData.aggregate_histogram('class').getInfo()
    validationInfo = validationData.aggregate_histogram('class').getInfo()

    traSeries = pd.Series(trainingInfo)
    valSeries = pd.Series(validationInfo)

    Points_concat = pd.DataFrame(pd.concat([traSeries, valSeries],ignore_index=True,axis=1))\
                    .rename(columns={0:'TraPoints',1:'ValPoints'}).rename({'0':'Sb','1':'Hb','2':'Dn'},axis='index')
    
    # Organize each matrix in separate excel sheets
    excelName = 'Mrx'+ smoothStr + imageID +'.xlsx'
    excel = pd.ExcelWriter(excelName, engine='xlsxwriter')

    Points_concat.to_excel(excel, sheet_name='Points', index=True, startrow=0)
    TM_concat.to_excel(excel, sheet_name='TrMrx', index=True, startrow=0)
    TA_concat.to_excel(excel, sheet_name='TrAcc', index=True, startrow=0)
    VM_concat.to_excel(excel, sheet_name='VaMrx', index=True, startrow=0)
    VA_concat.to_excel(excel, sheet_name='VaAcc', index=True, startrow=0)
    PU_concat.to_excel(excel, sheet_name='PU-Mrx', index=True, startrow=0)
    Kp_concat.to_excel(excel, sheet_name='Kappa', index=True, startrow=0)

    # Save matrices as .xlsx file:
    excel.save()
    print('Saved Matrices of '+imageID)

print('ALL IMAGES HAVE BEEN CLASSIFIED!')

Initiating...
Preparing image 20201124T160609_20201124T161304_T17RLM
Image masked...
Depth-Invariant index applied...
Training models and classifying...
Getting accuracies...
Exporting classified images to EE Assets...
Classified Image 1: 20201124T160609_20201124T161304_T17RLM_raw_CART submitted...
Classified Image 2: 20201124T160609_20201124T161304_T17RLM_raw_SVM submitted...
Classified Image 3: 20201124T160609_20201124T161304_T17RLM_raw_RF submitted...
Classified images submitted!
Saving matrices to working directory...
Saved Matrices of 20201124T160609_20201124T161304_T17RLM
Initiating...
Preparing image 20200203T160451_20200203T161119_T17RLM
Image masked...
Depth-Invariant index applied...
Training models and classifying...
Getting accuracies...
Exporting classified images to EE Assets...
Classified Image 1: 20200203T160451_20200203T161119_T17RLM_raw_CART submitted...
Classified Image 2: 20200203T160451_20200203T161119_T17RLM_raw_SVM submitted...
Classified Image 3: 20200203T160451